##### Used to analyze changes in NPV, PPV, and related clinical metrics under different risk-stratification thresholds.

In [ ]:
# -*- coding: utf-8 -*-
import copy
import re
import os
import math
import numpy as np
import pandas as pd
import shap
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pickle
from sklearn.model_selection import StratifiedKFold
import catboost
from catboost import *
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, auc
from statsmodels.stats.weightstats import ztest
from scipy.stats import norm
from imblearn.under_sampling import RandomUnderSampler
from sklearn.datasets import make_classification
from collections import Counter
from youden_for_risk_level import youden_index
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, classification_report

import warnings
warnings.filterwarnings("ignore")


# Modify data paths
db = 'SZBA'
cd = r'./'
search_path1 = cd + 'final_model/' + db + '/'  # model results

# Output path
save_path = cd + 'risk_level_divide/' + db + '/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

########################################################################################################
'''
Low risk: determined by NPV (close to 1)
Threshold 1
Medium risk: based on sample size (proportion) — originally Youden threshold
Threshold 2
High risk: determined by PPV
'''


def find_thr_1_by_npv(name):  # Find optimal threshold
    """
    Final CatBoost model Youden index
    """
    new_search_path = search_path1 + '/predict_5CV/' + name + '/'

    y_true_pred = pd.read_csv(new_search_path + 'predict_result.csv')

    y_true = y_true_pred['y_test']
    y_score = y_true_pred['y_tepred']
    youden_df, mj_val, mf1_val, auc = youden_index(y_true, y_score, pos_label=1, start=0, end=1)
    youden_dft = pd.DataFrame()
    for col in youden_df.columns:
        youden_dft[col] = youden_df[col].map("{:.3f}".format)

    print(youden_dft)

    # Threshold corresponding to NPV close to or equal to 1
    tmp = youden_df[youden_df['NPV'] >= 0.995]
    thr_val_1 = float(tmp['Thr'].values[-1])
    return thr_val_1


def find_thr_2_by_ppv(name):  # Find optimal threshold
    """
    Final CatBoost model Youden index
    """
    new_search_path = search_path1 + '/predict_5CV/' + name + '/'

    y_true_pred = pd.read_csv(new_search_path + 'predict_result.csv')

    y_true = y_true_pred['y_test']
    y_score = y_true_pred['y_tepred']

    youden_df, mj_val, mf1_val, auc = youden_index(y_true, y_score, pos_label=1, start=0, end=1)
    youden_dft = pd.DataFrame()
    for col in youden_df.columns:
        youden_dft[col] = youden_df[col].map("{:.3f}".format)

    # Select threshold where PPV ≥ 0.40
    tmp = youden_df[youden_df['PPV'] >= 0.40]
    thr_val_2 = float(tmp['Thr'].values[0])
    return thr_val_2


name = 'alldata'
###########################################  Risk stratification ############################################################
new_search_path = search_path1 + '/predict_5CV/' + name + '/'

y_true_pred = pd.read_csv(new_search_path + 'predict_result.csv')
print(y_true_pred['y_tepred'].describe())

# thr_val_best = 0.210  # Youden-selected threshold
thr_val_best = 0.175  # Youden-selected threshold
y_true_pred['is_risk'] = y_true_pred['y_tepred'].apply(lambda x: 1 if x > thr_val_best else 0)  # initial binary prediction

thr_val_1 = find_thr_1_by_npv(name)
thr_val_2 = find_thr_2_by_ppv(name)
thr_val_1 = round(thr_val_1, 3)
thr_val_2 = round(thr_val_2, 3)
print('thr_val_1,thr_val_2:', thr_val_1, ',', thr_val_2)
'''
thr_val_1,thr_val_2: 0.04 , 0.8
'''


########################################################################################################################
########################################### Separate 0/1 using original threshold ##################################################
'''
When two thresholds change (only one moves at a time),
evaluate trends in the metrics of the three risk levels.
'''


def index_calc(f, y_true, y_score, y_pred, pos_label=1):  # f: changing threshold
    fpr, tpr, thr = roc_curve(y_true, y_score, pos_label=pos_label)
    roc_auc = auc(fpr, tpr)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    _fpr, _tpr = 0.0, 0.0
    if fp + tn != 0:
        _fpr = fp / (fp + tn)
    if tp + fn != 0:
        _tpr = tp / (tp + fn)  # sensitivity
    ppv, npv = 0.0, 0.0
    if tp + fp != 0:
        ppv = tp / (tp + fp)
    if tn + fn != 0:
        npv = tn / (tn + fn)

    spec = 1.0 - _fpr  # specificity
    acc = (tp + tn) / float(tn + fp + fn + tp)
    jord_idx = _tpr + spec - 1  # Youden index
    f1 = 2 * ppv * _tpr / (ppv + _tpr)

    row = [f, acc, ppv, npv, _tpr, spec, jord_idx, f1, tp + fn, fp + tn, tp + fp, tn + fn, tp, fp, tn, fn]
    cols = "Thr\tACC\tPPV\tNPV\tSens(Rec/TPR)\tSpec\tYoudenIdx\tF1\tTrueBen\tTrueMal\tPredBen\tPredMal\tTP\tFP\tTN\tFN"
    columns = cols.split('\t')
    df = pd.DataFrame([row], columns=columns)
    return df


################################### Threshold 1 changes, threshold 2 fixed #################################################

# start1 = thr_val_1
# end1 = thr_val_best + 0.01  # modified
start1 = 0.01
end1 = 0.51

new_save_path1 = save_path + '/change_thr1/'
if not os.path.exists(new_save_path1):
    os.makedirs(new_save_path1)

# thrs_1 = list(np.arange(start1, end1, 0.005))
thrs_1 = list(np.arange(start1, end1, 0.01))

r0_results = pd.DataFrame()
r1_results = pd.DataFrame()
r2_results = pd.DataFrame()

for thr in thrs_1:
    # Initial predicted label: is_risk

    # subset1_0: predicted low risk (all 0)
    subset1_0 = y_true_pred[y_true_pred['y_tepred'] <= thr]
    r0 = index_calc(thr, subset1_0['y_test'], subset1_0['y_tepred'], subset1_0['is_risk'], pos_label=1)
    r0 = r0[['Thr', 'PPV', 'NPV', 'Sens(Rec/TPR)', 'Spec', 'YoudenIdx', 'TP', 'FP', 'TN', 'FN']]
    r0_t = pd.DataFrame()
    for col in r0.columns:
        if col == 'NPV':
            r0_t[col] = r0[col].map("{:.4f}".format)
        else:
            r0_t[col] = r0[col].map("{:.3f}".format)

    r0_t.insert(1, 'N', [len(subset1_0)])
    r0_t.insert(2, 'Proportion', ['{:.2f}%'.format(len(subset1_0) / len(y_true_pred) * 100)])
    r0_results = r0_results._append(r0_t)

    # subset1_1: medium risk (all 1; use proportion to evaluate clinical practical value)
    subset1_1 = y_true_pred[(y_true_pred['y_tepred'] > thr) & (y_true_pred['y_tepred'] <= thr_val_2)]
    r1 = index_calc(thr, subset1_1['y_test'], subset1_1['y_tepred'], subset1_1['is_risk'], pos_label=1)
    r1 = r1[['Thr', 'PPV', 'NPV', 'Sens(Rec/TPR)', 'Spec', 'YoudenIdx', 'TP', 'FP', 'TN', 'FN']]
    r1_t = pd.DataFrame()
    for col in r1.columns:
        r1_t[col] = r1[col].map("{:.3f}".format)
    r1_t.insert(1, 'N', [len(subset1_1)])
    r1_t.insert(2, 'Proportion', ['{:.2f}%'.format(len(subset1_1) / len(y_true_pred) * 100)])
    r1_results = r1_results._append(r1_t)

    # subset1_2: high risk (all 1; PPV-driven)
    subset1_2 = y_true_pred[y_true_pred['y_tepred'] > thr_val_2]
    r2 = index_calc(thr, subset1_2['y_test'], subset1_2['y_tepred'], subset1_2['is_risk'], pos_label=1)
    r2 = r2[['Thr', 'PPV', 'NPV', 'Sens(Rec/TPR)', 'Spec', 'YoudenIdx', 'TP', 'FP', 'TN', 'FN']]
    r2_t = pd.DataFrame()
    for col in r2.columns:
        r2_t[col] = r2[col].map("{:.3f}".format)
    r2_t.insert(1, 'N', [len(subset1_2)])
    r2_t.insert(2, 'Proportion', ['{:.2f}%'.format(len(subset1_2) / len(y_true_pred) * 100)])
    r2_results = r2_results._append(r2_t)

r0_results.to_csv(new_save_path1 + 'index_level0.csv')
r1_results.to_csv(new_save_path1 + 'index_level1.csv')
r2_results.to_csv(new_save_path1 + 'index_level2.csv')


################################### Threshold 2 changes, threshold 1 fixed #################################################

start2 = 0.180
end2 = 0.855

new_save_path2 = save_path + '/change_thr2/'
if not os.path.exists(new_save_path2):
    os.makedirs(new_save_path2)

thrs_2 = list(np.arange(start2, end2, 0.01))

change_thr2_r0 = pd.DataFrame()
change_thr2_r1 = pd.DataFrame()
change_thr2_r2 = pd.DataFrame()

for thr_end in thrs_2:

    # subset1_0: low-risk group (PPV irrelevant; focus on NPV)
    subset1_0 = y_true_pred[y_true_pred['y_tepred'] <= thr_val_1]
    r0 = index_calc(thr_end, subset1_0['y_test'], subset1_0['y_tepred'], subset1_0['is_risk'], pos_label=1)
    r0 = r0[['Thr', 'PPV', 'NPV', 'Sens(Rec/TPR)', 'Spec', 'YoudenIdx', 'TP', 'FP', 'TN', 'FN']]
    r0_t = pd.DataFrame()
    for col in r0.columns:
        if col == 'NPV':
            r0_t[col] = r0[col].map("{:.4f}".format)
        else:
            r0_t[col] = r0[col].map("{:.3f}".format)
    r0_t.insert(1, 'N', [len(subset1_0)])
    r0_t.insert(2, 'Proportion', ['{:.2f}%'.format(len(subset1_0) / len(y_true_pred) * 100)])
    change_thr2_r0 = change_thr2_r0._append(r0_t)

    # subset1_1: medium risk
    subset1_1 = y_true_pred[(y_true_pred['y_tepred'] > thr_val_1) & (y_true_pred['y_tepred'] <= thr_end)]
    r1 = index_calc(thr_end, subset1_1['y_test'], subset1_1['y_tepred'], subset1_1['is_risk'], pos_label=1)
    r1 = r1[['Thr', 'PPV', 'NPV', 'Sens(Rec/TPR)', 'Spec', 'YoudenIdx', 'TP', 'FP', 'TN', 'FN']]
    r1_t = pd.DataFrame()
    for col in r1.columns:
        r1_t[col] = r1[col].map("{:.3f}".format)
    r1_t.insert(1, 'N', [len(subset1_1)])
    r1_t.insert(2, 'Proportion', ['{:.2f}%'.format(len(subset1_1) / len(y_true_pred) * 100)])
    change_thr2_r1 = change_thr2_r1._append(r1_t)

    # subset1_2: high risk (PPV relevant)
    subset1_2 = y_true_pred[y_true_pred['y_tepred'] > thr_end]
    r2 = index_calc(thr_end, subset1_2['y_test'], subset1_2['y_tepred'], subset1_2['is_risk'], pos_label=1)
    r2 = r2[['Thr', 'PPV', 'NPV', 'Sens(Rec/TPR)', 'Spec', 'YoudenIdx', 'TP', 'FP', 'TN', 'FN']]
    r2_t = pd.DataFrame()
    for col in r2.columns:
        r2_t[col] = r2[col].map("{:.3f}".format)
    r2_t.insert(1, 'N', [len(subset1_2)])
    r2_t.insert(2, 'Proportion', ['{:.2f}%'.format(len(subset1_2) / len(y_true_pred) * 100)])
    change_thr2_r2 = change_thr2_r2._append(r2_t)

change_thr2_r0.to_csv(new_save_path2 + 'index_level0.csv')
change_thr2_r1.to_csv(new_save_path2 + 'index_level1.csv')
change_thr2_r2.to_csv(new_save_path2 + 'index_level2.csv')


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


count    8097.000000
mean        0.134804
std         0.195103
min         0.000176
25%         0.017250
50%         0.051330
75%         0.155865
max         0.993366
Name: y_tepred, dtype: float64
       Thr    ACC    PPV    NPV Sens(Rec/TPR)   Spec YoudenIdx     F1  \
0    0.000  0.032  0.032  0.000         1.000  0.000     0.000  0.063   
1    0.005  0.134  0.035  0.994         0.981  0.106     0.087  0.068   
2    0.010  0.195  0.038  0.995         0.973  0.169     0.143  0.073   
3    0.015  0.255  0.040  0.996         0.969  0.231     0.201  0.078   
4    0.020  0.307  0.043  0.995         0.958  0.286     0.244  0.082   
..     ...    ...    ...    ...           ...    ...       ...    ...   
195  0.975  0.968  0.583  0.968         0.027  0.999     0.026  0.051   
196  0.980  0.968  0.700  0.968         0.027  1.000     0.026  0.051   
197  0.985  0.968  0.800  0.968         0.015  1.000     0.015  0.030   
198  0.990  0.968  1.000  0.968         0.011  1.000     0.011  0.023  